### Non-Linear Optimazation (nlopt) short tutorial
---

Mission:  

$\min_{\mathbf{x}\in\mathbb{R}^2} \sqrt{x_2}$  
subject to $x_2 \geq 0, x_2 \geq (a_1 x_1 + b_1)^3, x_2 \geq (a_2 x_1 + b_2)^3 $

<div>
<img src="NLopt-example-constraints.png" width="600">
 </div>

[NLopt_Algorithms](https://nlopt.readthedocs.io/en/latest/NLopt_Algorithms/)

In [1]:
import nlopt
import numpy as np


In [2]:
def myfunc(x, grad):
    if grad.size > 0:
        grad[0] = 0.0  # partial derivative of the objective function to x[0]
        grad[1] = 0.5 / np.sqrt(x[1]) n# partial derivative of the objective function to x[1]
    return np.sqrt(x[1]) # Objective function

def myconstraint(x, grad, a, b):
    if grad.size > 0:
        grad[0] = 3 * a * (a*x[0] + b)**2  # derivative wrt x_0
        grad[1] = -1.0  # derivative wrt x_1
    return (a*x[0] + b)**3 - x[1]  # constraint: (a*x_0 + b)^3 - x_1 =<0

# `grad` is only for gradient-based methods!!
# if you a derivative-free optimazation algorithm, `grad` will always be `NULL` and you never need to compute any derivatives.

In [17]:
# Choose the optimizer:
# Notation used in NLOPT: NLOPT_{G,L}{N,D}_xxxx, where G/L denotes global/local optimization -
# and N/D denotes derivative-free/gradient-based algorithms, respectively.
# Examples:
# "LD" means local optimization, derivative/gradient-based
# "LN" means local optimization, no derivatives

# NLOPT_AUGLAG - Augmented Lagrangian.
opt = nlopt.opt(nlopt.LD_MMA, 2)  # Method of Moving Asymptotes

opt.set_lower_bounds([-float('inf'), 0])
opt.set_min_objective(myfunc)

opt.add_inequality_constraint(lambda x, grad: myconstraint(x,grad, 2, 0), 1e-8)  # 1e-8 is a tolerance for the constraint!
opt.add_inequality_constraint(lambda x, grad: myconstraint(x,grad, -1, 1), 1e-8)
opt.set_xtol_rel(1e-4) # tolerance for x parameter

In [19]:
x = opt.optimize([np.random.uniform(low=0, high=1100), np.random.uniform(low=0, high=1100)]) # starting values
minf = opt.last_optimum_value()
print("optimum at ", x[0], x[1])
print("minimum value = ", minf)
print("result code = ", opt.last_optimize_result())

optimum at  0.33333356358469857 0.29629601033128866
minimum value =  0.5443307912761216
result code =  4
